### Process to add detection marker and create video. by Hikaru Shishido ver1.00
 明視野画像に追跡点を示した動画を作成する

#### How to use 
- Change Folder path
- Put the following four files in the Folder specified above.
- ① rec.avi, ② log.csv, ③ parameter.txt

#### 使用方法 
- Folder pathを変更
- 上記で指定のFolderに以下3つのファイルを入れた状態にする
- ① rec.avi, ② log.csv, ③ parameter.txt

In [1]:
folder_paths = [
    # r"E:\_C.elegans\FigX\ana\20240508-185610tdTomato-10mW-1good",
    r"C:\Users\mlab-\Desktop\修論\FigX\ana\20240516-203245tdTomato-7mW-3"
]

In [6]:
import sys, cv2, csv, winsound

fps = 50
size = (640, 480)

for h in range(len(folder_paths)):
    mathching_video_filename = "rec detect " + str(fps) + " fps.avi"

    # video's path
    video_path = folder_paths[h] + "/" + "rec matching 50 fps.avi"
    # bright image csv file's path
    log_bright_path = folder_paths[h] + "/" + "log matching 50 fps.csv"
    # parameter txt file's path
    txt_path = folder_paths[h] + "/" + "parameter.txt"
    
    ################################################
    # read start_time from parameter txt file "parameter.txt"
    ################################################
    with open(txt_path, 'r') as file:
        lines = file.readlines()

    txt_data = {}
    for line in lines:
        if line.startswith("center:"):
            center_str = line.split(':')[1].strip()
            centerx, centery = map(int, center_str.strip('()').split(','))
            break
    
    # print(f"x: {centerx}, y: {centery}")
    print("1 / 2: Finihed to read center position from parameter.txt")

    ################################################
    # make video which added detection circle
    ################################################
    
    with open(log_bright_path, "r", encoding="shift_jis") as csv_bright:
        csvR_bright = csv.reader(csv_bright)
        header = next(csvR_bright)
        
        # 画像処理・動画保存
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("failed to open video")
            sys.exit()
        
        is_image, img = cap.read()
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        # writer = cv2.VideoWriter(folder_paths[h]+"/"+mathching_video_filename, fourcc, fps, (img.shape[1], img.shape[0]))
        writer = cv2.VideoWriter(folder_paths[h]+"/"+mathching_video_filename, fourcc, fps, size)
    
        n = 0
        for row in csvR_bright:
            
            if is_image:
                detection_x = int(row[7])
                detection_y = int(row[8])
                radius = int(row[18])
                asobi = int(row[19])
                # print(f"frame: {n}, x: {detection_x}, y: {detection_y}, radius: {radius}, asobi: {asobi}")
                
                img = cv2.circle(img, (detection_x, detection_y), radius*2, (255, 0, 255))
                img = cv2.circle(img, (detection_x, detection_y), asobi*2, (0, 255, 255))
                img = cv2.drawMarker(img, (detection_x, detection_y), (0, 255, 125))
                img = cv2.drawMarker(img, position=(centerx, centery), color=(255, 255, 0), markerType=cv2.MARKER_TILTED_CROSS, markerSize=10)
                
                writer.write(img)
                
            else:
                cap.release()
                writer.release()
                print("video-detect finished")
                break

            is_image, img = cap.read()
            n += 1
    
    cap.release()
    writer.release()
    print("2 / 2: Finihed to make video")

print("All process is finished")
winsound.Beep(1000, 200)

1 / 2: Finihed to read center position from parameter.txt
2 / 2: Finihed to make video
All process is finished
